<a href="https://colab.research.google.com/github/gratati/CV_segmentation/blob/main/%D0%A0%D0%B0%D1%81%D0%BA%D0%B0%D0%B4%D1%80%D0%BE%D0%B2%D0%BA%D0%B0_%D0%B2%D0%B8%D0%B4%D0%B5%D0%BE%D1%84%D0%B0%D0%BB%D0%BE%D0%B2_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
   #@title Подключение google-диска

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2

In [ ]:
#@title Константы

PATH = '/content/drive/MyDrive/'
VIDEO_PATH = '/content/drive/MyDrive/filename.mp4'
DATA_SET_PATH = '/content/drive/MyDrive/filename_data_set'
YOLO_IMGSZ = 640

# Папка на google-диске с "нарезанными" imgs
if not os.path.exists(DATA_SET_PATH):
  os.makedirs(DATA_SET_PATH)

In [ ]:
#@title Скрипт раскадровки видео

def get_imgs_from_custom_video(step: int = 3,                       # шаг сохранения img
                               video_path: str = VIDEO_PATH,        # путь к видео
                               data_set_path: str = DATA_SET_PATH,  # путь к папке с img
                               reshape: bool = False) -> None:      # изменить размер img под yolo(imgsz=640)

    vid_cap = cv2.VideoCapture(video_path)
    assert vid_cap.isOpened()

    img_w, img_h, fps_video, img_count = (vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH),
                                          vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT),
                                          vid_cap.get(cv2.CAP_PROP_FPS),
                                          vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f'Всего кадров: {img_count}. Ширина кадров: {img_w}. Высота кадров: {img_h}')
    print(f'FPS: {fps_video}')

    # решейп под обучение yolo
    if reshape:
      vid_cap.set(cv2.CAP_PROP_FRAME_WIDTH, YOLO_IMGSZ)
      vid_cap.set(cv2.CAP_PROP_FRAME_HEIGHT, YOLO_IMGSZ)

    success_read, frame = vid_cap.read()
    while success_read:
      current_frame = int(vid_cap.get(cv2.CAP_PROP_POS_FRAMES))

      # сохранение img
      cv2.imwrite(f'{data_set_path}/{current_frame:05d}.jpg', frame)

      # пропуск step кадров
      if step:
        next_img_index = current_frame + step
        if next_img_index < img_count:
          vid_cap.set(cv2.CAP_PROP_POS_FRAMES, next_img_index)
        else:
          break

      success_read, frame = vid_cap.read()

    vid_cap.release()


In [ ]:
#@title Раскадровка видео

get_imgs_from_custom_video(step=25,
                           video_path=VIDEO_PATH,
                           data_set_path=DATA_SET_PATH,
                           reshape=True)

### Раскадровка нескольких видео в папке

In [ ]:
import os
import cv2

PATH = '/content/drive/MyDrive/'
VIDEO_FOLDER = '/content/drive/MyDrive/foldername'
DATA_SET_PATH = '/content/drive/MyDrive/foldername_exit'
YOLO_IMGSZ = 640

def get_imgs_from_custom_video(step: int = 3,
                               video_path: str = VIDEO_FOLDER,
                               data_set_path: str = DATA_SET_PATH,
                               reshape: bool = False) -> None:
    print(f'Обработка видео: {video_path}')
    vid_cap = cv2.VideoCapture(video_path)
    if not vid_cap.isOpened():
        print(f"Ошибка: Не удалось открыть видео: {video_path}")
        return

    img_w = vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    img_h = vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps_video = vid_cap.get(cv2.CAP_PROP_FPS)
    img_count = vid_cap.get(cv2.CAP_PROP_FRAME_COUNT)
    print(f'Всего кадров: {img_count}. Ширина кадров: {img_w}. Высота кадров: {img_h}')
    print(f'FPS: {fps_video}')

    if reshape:
        vid_cap.set(cv2.CAP_PROP_FRAME_WIDTH, YOLO_IMGSZ)
        vid_cap.set(cv2.CAP_PROP_FRAME_HEIGHT, YOLO_IMGSZ)

    frame_count = 0
    video_folder_name = os.path.splitext(os.path.basename(video_path))[0]
    video_data_set_path = os.path.join(data_set_path, video_folder_name)
    if not os.path.exists(video_data_set_path):
        os.makedirs(video_data_set_path)

    while True:
        ret, frame = vid_cap.read()
        if not ret:
            break

        frame_count += 1

        # Сохранение каждого 50-го кадра
        if frame_count % step == 0:
            frame_path = os.path.join(video_data_set_path, f"{frame_count:05d}.jpg")
            cv2.imwrite(frame_path, frame)
            print(f'Сохранено: {frame_path}')

    vid_cap.release()

def process_videos_in_folder(folder_path, step=3, reshape=False):
    video_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if not video_files:
        print(f"Ошибка: Нет видеофайлов в папке: {folder_path}")
        return

    for video_file in video_files:
        video_path = os.path.join(folder_path, video_file)
        get_imgs_from_custom_video(step=step, video_path=video_path, reshape=reshape)

# Обработка всех видео в указанной папке
process_videos_in_folder(VIDEO_FOLDER, step=3, reshape=True)


###Собираем все изображения в одну папку

In [ ]:
import os
import shutil
import re

# Исходная папка, содержащая все другие папки
source_dir = '/content/drive/MyDrive/foldername'

# Папка для сохранения изображений
output_dir = '/content/drive/MyDrive/foldername_all'

# Создаем папку для сохранения изображений, если она не существует
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Функция для перемещения и переименования файлов
def move_and_rename_files(src, dst):
    file_counts = {}
    for filename in os.listdir(src):
        src_file = os.path.join(src, filename)
        if os.path.isfile(src_file):
            # Проверяем, является ли файл изображением
            if filename.endswith('.jpg') or filename.endswith('.png'):
                # Используем имя файла без расширения в качестве ключа для подсчета
                base_name = os.path.splitext(filename)[0]
                if base_name in file_counts:
                    file_counts[base_name] += 1
                else:
                    file_counts[base_name] = 1
                new_filename = f"{base_name}_{file_counts[base_name]:04d}.{os.path.splitext(filename)[1]}"
                dst_file = os.path.join(dst, new_filename)
                shutil.move(src_file, dst_file)

# Рекурсивная функция для обхода папок и подпапок
def traverse_directories(directory):
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path):
            traverse_directories(item_path)
            move_and_rename_files(item_path, output_dir)

# Запускаем обход папок и сохранение изображений
traverse_directories(source_dir)

##Подсчет колличества изображений в итоговой папке

In [ ]:
import os

# Путь к папке с изображениями
image_folder = '/content/drive/MyDrive/foldername_all'

# Получение списка всех файлов в папке
files = os.listdir(image_folder)

# Фильтрация списка, оставляя только файлы с расширениями изображений
image_files = [f for f in files if f.endswith('.png') or f.endswith('.jpg') or f.endswith('.jpeg')]

# Подсчет количества изображений
num_images = len(image_files)

print(f"Количество изображений в папке: {num_images}")